In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as F
import torch.optim as optim

import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
from libs import data as dt, neuronshap as ns, sim
from cfgs.fedargs import *

from fairlearn.metrics import (
    demographic_parity_difference,
    demographic_parity_ratio,
    equalized_odds_difference,
    equalized_odds_ratio,
    false_negative_rate,
    false_positive_rate,
    true_negative_rate,
    true_positive_rate,
)
from libs.helpers.finance import bin_hours_per_week
from libs.helpers.metrics import (
    conditional_demographic_parity_difference,
    conditional_demographic_parity_ratio,
)
from libs.helpers.plot import group_box_plots

In [2]:
names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "salary",
]

In [3]:
def clean_string(s):
    """
    Helper function that strips leading / trailing whitespace, lower
    cases, and replaces hyphens with underscores.
    """
    return s.strip().lower().replace("-", "_")


def parse_native_country(country):
    """
    Group countries other than United-States and Mexico into single
    "other" category"
    """
    country = clean_string(country)
    if country == "united_states" or country == "mexico":
        return country
    return "other"

In [4]:
train = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
        header=None,
        na_values=[" ?"],
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        salary=lambda df: (df.salary == " >50K").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [5]:
test = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
        header=None,
        na_values=[" ?"],
        skiprows=1,
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        # note extra '.' in test set not present in train set
        salary=lambda df: (df.salary == " >50K.").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [6]:
assert set(train.education) == set(test.education)
assert set(train.race) == set(test.race)
assert set(train.relationship) == set(test.relationship)
assert set(train.marital_status) == set(test.marital_status)

In [7]:
one_hot_features = [
    "workclass",
    "education",
    "occupation",
    "race",
    "relationship",
    "marital_status",
    "native_country",
]

cts_features = ["age", "capital_gain", "capital_loss", "hours_per_week"]

binary_features = ["sex", "salary"]

In [8]:
train["race"].value_counts()

race
white                 25933
black                  2817
asian_pac_islander      895
amer_indian_eskimo      286
other                   231
Name: count, dtype: int64

In [9]:
train_df = pd.concat(
    [train, pd.get_dummies(train.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

test_df = pd.concat(
    [test, pd.get_dummies(test.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

In [10]:
assert train_df.columns.tolist() == test_df.columns.tolist()

In [11]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [12]:
data_dir = "../../data/adult"

In [13]:
original_features = cts_features + one_hot_features + binary_features

train_df[original_features].to_csv("../../data/adult/train.csv", index=False)
val_df[original_features].to_csv("../../data/adult/val.csv", index=False)
test_df[original_features].to_csv("../../data/adult/test.csv", index=False)

In [14]:
ss = StandardScaler()

train_df[cts_features] = ss.fit_transform(train_df[cts_features])
val_df[cts_features] = ss.transform(val_df[cts_features])
test_df[cts_features] = ss.transform(test_df[cts_features])

/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/sklearn/utils/validation.py:753: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/sklearn/utils/validation.py:591: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/sklearn/utils/validation.py:600: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/sklearn/utils/validation.py:753: FutureWarning: is_sparse is deprecated and will be

In [15]:
train_df.drop(columns=one_hot_features).to_csv("../../data/adult/train-one-hot.csv", index=False)
val_df.drop(columns=one_hot_features).to_csv("../../data/adult/val-one-hot.csv", index=False)
test_df.drop(columns=one_hot_features).to_csv("../../data/adult/test-one-hot.csv", index=False)

In [16]:
train = pd.read_csv("../../data/adult/train.csv")
val = pd.read_csv("../../data/adult/val.csv")
test = pd.read_csv("../../data/adult/test.csv")

train_oh = pd.read_csv("../../data/adult/train-one-hot.csv")
val_oh = pd.read_csv("../../data/adult/val-one-hot.csv")
test_oh = pd.read_csv("../../data/adult/test-one-hot.csv")

In [17]:
train_oh.head()

,age,sex,capital_gain,capital_loss,hours_per_week,salary,workclass_federal_gov,workclass_local_gov,workclass_private,workclass_self_emp_inc,...,marital_status_divorced,marital_status_married_af_spouse,marital_status_married_civ_spouse,marital_status_married_spouse_absent,marital_status_never_married,marital_status_separated,marital_status_widowed,native_country_mexico,native_country_other,native_country_united_states
0,1.108936,1,-0.147741,-0.218133,2.416833,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,0.805386,1,-0.147741,-0.218133,2.416833,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,-0.788255,1,-0.147741,-0.218133,-0.079269,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
3,1.640150,0,-0.147741,-0.218133,-0.079269,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
4,1.108936,1,-0.147741,-0.218133,-0.079269,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1


In [18]:
test_oh.head(5)

,age,sex,capital_gain,capital_loss,hours_per_week,salary,workclass_federal_gov,workclass_local_gov,workclass_private,workclass_self_emp_inc,...,marital_status_divorced,marital_status_married_af_spouse,marital_status_married_civ_spouse,marital_status_married_spouse_absent,marital_status_never_married,marital_status_separated,marital_status_widowed,native_country_mexico,native_country_other,native_country_united_states
0,-1.015917,1,-0.147741,-0.218133,-0.079269,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
1,-0.029378,1,-0.147741,-0.218133,0.752765,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
2,-0.788255,1,-0.147741,-0.218133,-0.079269,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0.425948,1,0.872159,-0.218133,-0.079269,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
4,-0.332929,1,-0.147741,-0.218133,-0.911303,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1


In [19]:
#https://github.com/ritvikkhanna09/Census-classifier-comparison

In [20]:
'''
race_amer_indian_eskimo               15060 non-null  int64  
race_asian_pac_islander               15060 non-null  int64  
race_black                            15060 non-null  int64  
race_other                            15060 non-null  int64  
race_white
 
'''

mr_dh_oh = test_oh.loc[(test_oh["race_asian_pac_islander"] == 1) | (test_oh["race_white"] == 1)]
mr_dh_oh = mr_dh_oh.head(200)
fmr_dh_oh = test_oh.loc[(test_oh["race_amer_indian_eskimo"] == 1) | (test_oh["race_black"] == 1) | (test_oh["race_other"] == 1)]
fmr_dh_oh = fmr_dh_oh.head(200)


m_dh_oh = test_oh.loc[test_oh["sex"] == 1]
m_dh_oh = m_dh_oh.head(200)
fm_dh_oh = test_oh.loc[test_oh["sex"] == 0]
fm_dh_oh = fm_dh_oh.head(200)

In [21]:
X_train = train_oh.drop(columns="salary").values
Y_train = train_oh['salary'].values
X_test = test_oh.drop(columns="salary").values
Y_test = test_oh['salary'].values
X_m = m_dh_oh.drop(columns="salary").values
Y_m = m_dh_oh['salary'].values
X_fm = fm_dh_oh.drop(columns="salary").values
Y_fm = fm_dh_oh['salary'].values

X_mr = mr_dh_oh.drop(columns="salary").values
Y_mr = mr_dh_oh['salary'].values
X_fmr = fmr_dh_oh.drop(columns="salary").values
Y_fmr = fmr_dh_oh['salary'].values

#creating torch dataset and loader using original dataset. 
#to use resampled dataset, replace ex. xtrain with xtrain_over etc.
train_data = torch.utils.data.TensorDataset(torch.tensor(X_train).float(), torch.tensor(Y_train).long())
test_data = torch.utils.data.TensorDataset(torch.tensor(X_test).float(), torch.tensor(Y_test).long())
m_data = torch.utils.data.TensorDataset(torch.tensor(X_m).float(), torch.tensor(Y_m).long())
fm_data = torch.utils.data.TensorDataset(torch.tensor(X_fm).float(), torch.tensor(Y_fm).long())

mr_data = torch.utils.data.TensorDataset(torch.tensor(X_mr).float(), torch.tensor(Y_mr).long())
fmr_data = torch.utils.data.TensorDataset(torch.tensor(X_fmr).float(), torch.tensor(Y_fmr).long())

train_loader = torch.utils.data.DataLoader(train_data,batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data))
m_loader = torch.utils.data.DataLoader(m_data, batch_size=1)
fm_loader = torch.utils.data.DataLoader(fm_data, batch_size=1)
mr_loader = torch.utils.data.DataLoader(mr_data, batch_size=1)
fmr_loader = torch.utils.data.DataLoader(fmr_data, batch_size=1)

In [22]:
class BasicNet(torch.nn.Module):
    
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_features = num_features
        self.num_classes = num_classes
        self.layers = 0
        
        self.lin1 = torch.nn.Linear(self.num_features,  150)        
        self.lin2 = torch.nn.Linear(50, 50)        
        self.lin3 = torch.nn.Linear(50, 50)
        
        self.lin4 = torch.nn.Linear(150, 150) 
        
        self.lin5 = torch.nn.Linear(50, 50)        
        self.lin6 = torch.nn.Linear(50, 50)
        self.lin10 = torch.nn.Linear(150, self.num_classes)
        
        self.prelu = torch.nn.PReLU()
        self.dropout = torch.nn.Dropout(0.25)

    def forward(self, xin):
        self.layers = 0
        
        x = F.relu(self.lin1(xin))
        self.layers += 1
        
        #x = F.relu(self.lin2(x))
        #self.layers += 1
        for y in range(16):
            x = F.relu(self.lin4(x)) 
            self.layers += 1
           
        x = self.dropout(x)
        
        x = F.relu(self.lin10(x)) 
        self.layers += 1
        return x

In [23]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    
    for inputs, target in train_loader:
      
        #inputs, target = inputs.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_fn(output, target.long())
        # Backprop
        loss.backward()
        optimizer.step()
        ###

In [24]:
def test(model, test_loader):
    model.eval()
    
    test_loss = 0
    correct = 0
    test_size = 0
    
    with torch.no_grad():
      
        for inputs, target in test_loader:
            
            #inputs, target = inputs.to(device), target.to(device)
            
            output = model(inputs)
            test_size += len(inputs)
            test_loss += test_loss_fn(output, target.long()).item() 
            pred = output.max(1, keepdim=True)[1] 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= test_size
    accuracy = correct / test_size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, test_size,
        100. * accuracy))
    
    return test_loss, accuracy

In [25]:
model = BasicNet(63, 2)
test_accuracy = []
train_loss = []
nbr_epochs = 5
lr = 0.0025# 
weight_decay = 0

# Surrogate loss used for training
loss_fn = torch.nn.CrossEntropyLoss()
test_loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')

optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=lr ,weight_decay=weight_decay)
#optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)

print('Training beginning...')
#start_time = time.time()

for epoch in range(1, nbr_epochs+1):
    print('Epoch ', epoch, ':')
    train(model, train_loader, optimizer, epoch)
    loss, acc = test(model, test_loader)
    
    # save results every epoch
    test_accuracy.append(acc)
    train_loss.append(loss)
    
#end_time = time.time()
#print('Training on ' + str(nbr_epochs) + ' epochs done in ', str(end_time-start_time),' seconds')

Training beginning...
Epoch  1 :

Test set: Average loss: 0.3499, Accuracy: 12565/15060 (83%)

Epoch  2 :

Test set: Average loss: 0.3236, Accuracy: 12789/15060 (85%)

Epoch  3 :

Test set: Average loss: 0.3358, Accuracy: 12815/15060 (85%)

Epoch  4 :

Test set: Average loss: 0.3380, Accuracy: 12728/15060 (85%)

Epoch  5 :

Test set: Average loss: 0.3453, Accuracy: 12789/15060 (85%)



In [26]:
with torch.no_grad():
    for inputs, target in test_loader:
        outputs = model(inputs)
        pred = outputs.max(1, keepdim=True)[1] 
        correct = pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(inputs)
        print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, len(inputs), 100. * accuracy))

Y_prob = F.softmax(outputs, dim=1)[:, 1]
Y_pred = outputs.max(1, keepdim=True)[1]

print(sum(Y_test), sum(Y_pred), sum(pred))

test = pd.read_csv("../../data/adult/test.csv")


Accuracy: 12789/15060 (85%)

3700 tensor([2645]) tensor([2645])


<h1>Demographic Parity</h1>

<h2>Distribution of scores by sex</h2>

In [27]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test.sex,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test.sex,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.146
Demographic parity ratio: 0.344


/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

<h2>Distribution of scores by race</h2>

In [28]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test.race,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test.race,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.207
Demographic parity ratio: 0.226


/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

<h1>Conditional Demographic Parity</h1>

<h2>Distribution of scores by sex and hours worked per week</h2>

In [29]:
test_hpw_enum = test.hours_per_week.map(bin_hours_per_week)

cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

Conditional demographic parity difference: 0.142
Conditional demographic parity ratio: 0.418


/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

<h2>Distribution of scores by race and hours worked per week</h2>

In [30]:
cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test.race, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test.race, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

Conditional demographic parity difference: 0.240
Conditional demographic parity ratio: 0.122


/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)


<h1>Equalised Odds</h1>

<h2>Distribution of scores by sex for high and low earners</h2>

In [31]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test.sex,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test.sex,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.052
Equalised odds ratio: 0.294


/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

<h2>Distribution of scores by race for high and low earners</h2>

In [32]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test.race,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test.race,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

Equalised odds difference: 0.394
Equalised odds ratio: 0.225


/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

<h1>Shapley based Neuron Pruning for Fairness</h1>

In [94]:
m_shapley_values = ns.calculate_shapley_values_fa(model, m_loader, 200)
print(m_shapley_values)
fm_shapley_values = ns.calculate_shapley_values_fa(model, fm_loader, 200)
print(fm_shapley_values)

[1.5041212e+00 2.4817765e+00 1.3415010e+00 ... 2.9708777e+03 3.1958005e+03
 0.0000000e+00]
[1.9787655e+00 0.0000000e+00 1.6109374e+00 ... 2.7959749e+03 2.7285452e+03
 0.0000000e+00]


In [131]:
diff_shap_values = m_shapley_values - fm_shapley_values
max_diff_shap_values_ind = np.argpartition(diff_shap_values, -180)[-180:]
diff_shap_values[max_diff_shap_values_ind]

array([  7.833032 ,   7.876215 ,   7.9634695,   7.927437 ,   8.067585 ,
         8.073025 ,   8.117163 ,   8.129124 ,   8.186088 ,   8.417229 ,
         8.341816 ,   8.605679 ,   8.355246 ,   8.608654 ,   8.26741  ,
         8.664399 ,   8.431799 ,   8.490686 ,   8.561333 ,   8.398476 ,
         8.460441 ,   8.472932 ,   8.214056 ,   8.580116 ,   8.236263 ,
         8.234024 ,   8.692789 ,   8.715572 ,   8.550116 ,   8.3269825,
         8.559199 ,   8.453468 ,   8.403866 ,   8.798553 ,   9.968555 ,
        18.316803 ,   9.08692  ,  12.585687 ,   9.190641 ,   9.190882 ,
        15.092774 ,  17.406548 ,  10.976928 ,  10.880795 ,  13.14946  ,
       133.65155  ,   9.363241 ,   9.081495 ,  78.469376 ,  66.94806  ,
        49.51053  ,  19.886297 ,  38.07077  ,  12.363564 ,  20.131838 ,
        60.348694 , 206.8249   ,  17.916992 ,  20.754446 ,  29.946747 ,
         9.650795 ,  29.745697 ,  10.484169 ,  86.63287  ,  49.99109  ,
        15.131455 ,  11.331375 ,  42.858063 ,  27.041035 ,   9.8

In [132]:
model_arr, model_slist = sim.get_net_arr(model)
model_arr[max_diff_shap_values_ind] = 0
updated_model = sim.get_arr_net(model, model_arr, model_slist)

In [133]:
mr_shapley_values = ns.calculate_shapley_values_fa(updated_model, mr_loader, 200)
print(mr_shapley_values)
fmr_shapley_values = ns.calculate_shapley_values_fa(updated_model, fmr_loader, 200)
print(fmr_shapley_values)

[0.9571159  0.7546502  0.60250115 ... 0.         0.         0.        ]
[1.0242783  0.90579945 0.8204116  ... 0.         0.         0.        ]


In [134]:
diff_shap_values = mr_shapley_values - fmr_shapley_values
max_diff_shap_values_ind = np.argpartition(diff_shap_values, -180)[-180:]
diff_shap_values[max_diff_shap_values_ind]

array([0.30766624, 0.3166418 , 0.31166366, 0.31579694, 0.31709933,
       0.3111902 , 0.32348564, 0.32562137, 0.45351303, 0.49607873,
       0.6769391 , 0.33093417, 1.0809298 , 1.861874  , 0.69532514,
       0.3625003 , 0.61400825, 0.43230808, 0.42297518, 0.5207472 ,
       0.44368935, 0.37601328, 2.19975   , 0.41658685, 0.642466  ,
       0.52253854, 0.718484  , 6.7179236 , 0.9163888 , 1.6216111 ,
       0.35440785, 0.3591255 , 0.58270395, 0.4980197 , 0.53933996,
       2.8184998 , 0.70947737, 0.34556627, 1.0599396 , 4.911327  ,
       0.38241255, 0.4266746 , 0.37761527, 0.5973525 , 0.36493754,
       1.4839363 , 0.33277994, 0.8232641 , 0.42966387, 0.435897  ,
       1.3739799 , 0.4534791 , 6.10461   , 0.5126959 , 5.903162  ,
       0.47300494, 0.38038883, 0.8378283 , 0.98552513, 0.35478684,
       0.34736577, 0.63083935, 1.5799271 , 1.3915569 , 0.5241727 ,
       0.5297224 , 0.38919735, 0.7698692 , 0.7802572 , 0.56425345,
       0.39927676, 1.0229098 , 0.36980385, 1.26927   , 0.59419

In [135]:
model_arr, model_slist = sim.get_net_arr(updated_model)
model_arr[max_diff_shap_values_ind] = 0
updated_model_2 = sim.get_arr_net(model, model_arr, model_slist)

In [136]:
with torch.no_grad():
    for inputs, target in test_loader:
        outputs = updated_model_2(inputs)
        pred = outputs.max(1, keepdim=True)[1] 
        correct = pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / len(inputs)
        print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, len(inputs), 100. * accuracy))
        

Y_prob = F.softmax(outputs, dim=1)[:, 1]
Y_pred = outputs.max(1, keepdim=True)[1]

print(sum(Y_test), sum(Y_pred), sum(pred))

test = pd.read_csv("../../data/adult/test.csv")


Accuracy: 11441/15060 (76%)

3700 tensor([81]) tensor([81])


In [137]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test.sex,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test.sex,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.004
Demographic parity ratio: 0.395


/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

In [138]:
test_hpw_enum = test.hours_per_week.map(bin_hours_per_week)

cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test.sex, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

Conditional demographic parity difference: 0.005
Conditional demographic parity ratio: 0.421


/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

In [139]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test.sex,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test.sex,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

ZeroDivisionError: float division by zero

In [140]:
dpd = demographic_parity_difference(
    Y_test, Y_pred, sensitive_features=test.race,
)
dpr = demographic_parity_ratio(
    Y_test, Y_pred, sensitive_features=test.race,
)

print(f"Demographic parity difference: {dpd:.3f}")
print(f"Demographic parity ratio: {dpr:.3f}")

Demographic parity difference: 0.010
Demographic parity ratio: 0.000


/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

In [141]:
test_hpw_enum = test.hours_per_week.map(bin_hours_per_week)

cdpd = conditional_demographic_parity_difference(
    Y_test, Y_pred, test.race, test_hpw_enum,
)
cdpr = conditional_demographic_parity_ratio(
    Y_test, Y_pred, test.race, test_hpw_enum,
)

print(f"Conditional demographic parity difference: {cdpd:.3f}")
print(f"Conditional demographic parity ratio: {cdpr:.3f}")

/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

Conditional demographic parity difference: 0.014
Conditional demographic parity ratio: 0.000


/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

In [142]:
eod = equalized_odds_difference(
    Y_test, Y_pred, sensitive_features=test.race,
)
eor = equalized_odds_ratio(
    Y_test, Y_pred, sensitive_features=test.race,
)

print(f"Equalised odds difference: {eod:.3f}")
print(f"Equalised odds ratio: {eor:.3f}")

/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.isscalar(x) else np.nan)
/Users/hkasyap/anaconda3/envs/shap/lib/python3.10/site-packages/fairlearn/metrics/_disaggregated_result.py:235: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mf = mf.applymap(lambda x: x if np.i

ZeroDivisionError: float division by zero